In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

Tue Jun 11 19:41:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:19:00.0 Off |                  Off |
|  0%   48C    P8             38W /  450W |   18410MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Get PNG file list

In [9]:
import os
import numpy as np
from tqdm import tqdm

def find_png_files(directory):
    png_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.png'):
                full_path = os.path.join(root, file)
                png_files.append(full_path)
    return png_files

# Example usage
directory = '/data/sd-dataset/jisoo_png'  # Assuming the user wants to search this directory
png_files = find_png_files(directory)
np.random.shuffle(png_files)
len(png_files)

27

### Load BLIP2

In [ ]:
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

processor2 = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model2 = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards:  50%|█████████         | 1/2 [00:02<00:02,  2.45s/it]

### Image to Text

In [ ]:
def image_to_text(image_file, text=None):
    raw_image = Image.open(image_file).convert('RGB')

    inputs = processor2(images=raw_image, text=text, return_tensors="pt").to('cuda:0', torch.float16)

    generated_ids = model2.generate(**inputs, max_length=50)
    generated_text = processor2.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    return generated_text

In [ ]:
from tqdm import tqdm
import os

prefix = 'jisoo'
for image_file in tqdm(png_files):
    save_file = image_file[:-3] + 'txt'
    prompt = prefix + ', ' + image_to_text(image_file)
    with open(save_file, 'w') as f:
        f.write(prompt)
